# ___Timezone___

#### 한가지 ___Timezone___ 안에 데이터만 다룬다면 몰라도 괜찮다 하지만 ___Timezone___이 섞이게 된다면 꼭 알아야한다
#### 가령 ___미국 Rawdata___와 ___한국 Rawdata___의 시간차를 고려하지 않고 Merge하여 Analysis 한다면 그 이후 작업은 전부 잘못된 데이터를 다루는 것이다
#### 따라서 ___Datetime Index___ 자체에 이 데이터는 어떤 ___Timezone___의 데이터인지 지정해주는 습관이 중요하다
#### 가령 구글 G-mail은 해외에서 열면 해당 국가 ___Timezone___으로 바껴서 나온다

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as display

In [ ]:
from datetime import datetime
from datetime import timezone
    # 같은 datetime Package 안에 timezone Package가 있다

In [ ]:
timezone.utc    
    # timezone Package 안에는 utc라는 timezone 밖에 없다 (in Python)
    # 따라서 utc에 +9 해주면 KST(Korea Standard Time)이 된다
    # 그런 작업을 해주어야 하기 때문에 불편하다

In [ ]:
datetime(2019, 1, 1)    
    # timezone unawared datetime

In [ ]:
datetime(2019, 1, 1, tzinfo=timezone.utc)
    # timezone awared datetime (같은 2019-01-01 이지만 utc 시간이 명시되어있다)

In [ ]:
# 위의 datetime을 KST로 바꾸거나 특정 시간대와의 차이를 알려면 Extra Package를 쓴다
# conda[pip] install pytz / 많이 사용되는 Package이고 미래에는 Python Standard Library에 들어올 수도 있다
import pytz

In [ ]:
pytz.UTC   
    # = pytz.utc
pytz.all_timezones   
    # 지정할 수 있는 Hyper Parameter(location)이 모두 나온다
pytz.timezone('Asia/Seoul')    
    # 한국의 timezone

In [ ]:
datetime(2019, 1, 1, tzinfo=timezone.utc) == pytz.UTC.localize(datetime(2019, 1, 1))
    # timezone Package 에서는 UTC 말고는 정의하기 힘들다 

In [ ]:
KST = pytz.timezone('Asia/Seoul')
KST.localize(datetime(2019, 1, 1)) == datetime(2019, 1, 1, tzinfo=timezone.utc)   
    # KST 가 UTC + 9h 라서 False

In [62]:
# 한 가지 조심해야할 것! (.replce(tzinfo='') 불가!)
my_date = datetime(2019, 1, 1)

# datetime Object의 시간을 바꿀 때 많이 사용하는 방법
my_date.replace(year=2018)

my_date.replace(tzinfo=KST)
    # replace()로 tzinfo를 바꾸면 +9이 아니라 +8:28이 된다
    # 그래서 timezone에 localize를 한 다음에 datetime 객체를 넘겨준 다음에 return 받아야합니다
my_date_kst = KST.localize(my_date)
my_date_kst

datetime.datetime(2019, 1, 1, 0, 0, tzinfo=<DstTzInfo 'Asia/Seoul' KST+9:00:00 STD>)

In [65]:
# timezone을 설정하고 난 후 바꿀 수도 있다. 시간을 바꾸지 않고 timezone만 바꾸는 것이 아닌 시간과 timezone 둘 다 바꿀 때,
my_date_kst.astimezone(pytz.utc)    
    # datetime.datetime(2018, 12, 31, 15, 0, tzinfo=<UTC>)
    # 원래 1월 1일 00시 였는데 12월 31일 15시로 -9시간 되며 UTC 시간으로 바뀌었다

my_date_utc = my_date_kst.astimezone(pytz.utc)
my_date_utc == my_date_kst    # True
    # my_date_kst는 2019년 01월 01일 00시인데 timezone이 KST 이다
    # my_date_utc는 2018년 12월 31일 15시인데 timezone이 UTC 이다
    # datetime에 timezone을 저장해놓는 순간 convertion 해서 모든 시간으로 바꿀 수 있다

## ___datetime___ 관련 함수들: ___strftime()___, ___today()___, ___weekday()___, ...

In [91]:
# Date Parsing 할 때, strptime(String Parse Time: Parsing a string representing a time according to a format)이 있었다

# 이것은 시간을 저장할 때 사용하는 strftime(String Format Time)
my_date_kst.strftime("%Y-%m-%d %H:%M:%S %Z")    
    # %Z 은 timezone이다
my_date_kst.astimezone(pytz.utc).strftime("%Y-%m-%d %H:%M:%S %Z")    
    # '2018-12-31 15:00:00 UTC'
    # strftime.org 라는 사이트에서 Code 를 참고하자 (범용이라서 모든 Language에서 똑같다)
    
datetime.today()
    # system의 location을 읽어서 시간을 결정한다(해당 PC에 설정된 timezone에 따라 달라진다)
datetime.utcnow().astimezone(KST)
    # 그래서 일단 UTC로 시간을 받아온 담음 원하는 timezone으로 바꾸는 이 Code를 더 선호한다

today = datetime.today()
today.weekday()
    # 오늘이 무슨 요일인지 알고 싶을 때 (0: 월요일, 1: 화요일, 2: 수요일, ..., 6: 일요일)

# e.g) 가장 최근의 금요일을 알아내보자
# Hint: 오늘에서 일주일을 뺀다음 오늘이 지난 금요일에서 며칠이 지났는지 offset을 더해주면 된다

def my_last_friday():
    today = KST.localize(datetime.today())
    target_day = today
    for i in range(7):
        target_day = target_day.replace(day=today.day-i)
        if target_day.weekday() == 4:
            return target_day
        
def last_friday():
    today = KST.localize(datetime.today())
    offset = 4 - today.weekday() % 4
    a_week_ago = today.replace(day=today.day-7)
    last_friday = a_week_ago.replace(day=a_week_ago.day+offset)
    return last_friday
    # 이 함수에는 문제가 하나 있다 
    # 월 초에 이 함수를 쓰면 오늘이 7일 미만이면 day=today.day-7 in line4 코드에 의해 음수값이 나와 에러가 난다
    # 월 말에 이 함수를 쓰면 (가령 30일이라면 offset을 더하면 35일이 딘다) 에러가 난다
    # 그런 문제를 해결하기 위해서 timedelta가 있다

## ___timedelta()___

In [91]:
from datetime import timedelta
pytz.utc.localize(my_date) - KST.localize(my_date)
    # datetime.timedelta(seconds=32400) >>> timedelta에 초로 저장된다 (32,400초는 9시간이다)
pytz.utc.localize(my_date) + timedelta(seconds=32400)
    # datetime.datetime(2019, 1, 1, 9, 0, tzinfo=<UTC>) >>> 9시간 늘어났다. UTC가 9시니 KST로 바꾸면 -9하면 된다
    # 월을 넘어가거나 앞당기게 되면 그것도 같이 계산해준다

def better_last_friday():
    today = KST.localize(datetime.today())
    offset = 4 - today.weekday() % 4
    a_week_ago = today - timedelta(weeks=1)
        # instead of >>> a_week_ago = today.replace(day=today.day-7)
    last_friday = a_week_ago + timedelta(days=offset)
        # instead of >>> last_friday = a_week_ago.replace(day=a_week_ago.day+offset)
    return last_friday

## ___Time Series Data Visualization___ (시계열 데이터 시각화)


In [96]:
# 다시 애플..
df_apple = pd.read_csv('data/apple_stock.csv', index_col='Date', parse_dates=True)
    # parse_dates=True 하면 'Date' Column(object type)을 datetime type로 Parsing 한다
df_apple.head() 

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2009-12-31,30.478571,30.080000,30.447144,30.104286,88102700.0,20.159719
2010-01-04,30.642857,30.340000,30.490000,30.572857,123432400.0,20.473503
2010-01-05,30.798571,30.464285,30.657143,30.625713,150476200.0,20.508902
2010-01-06,30.747143,30.107143,30.625713,30.138571,138040000.0,20.182680
2010-01-07,30.285715,29.864286,30.250000,30.082857,119282800.0,20.145369
